In [15]:
import pandas as pd
import geopandas as gpd

In [16]:
bus_data = pd.read_csv("../data/iett_otobus_duraklar.csv")

In [17]:
bus_data.head()

,geometry
0,POINT (29.2763499999941 40.8183030005252)
1,POINT (29.266183999994 40.8140130005245)
2,POINT (29.3429089999946 40.8837970005355)
3,POINT (29.3517150000085 40.8281299991206)
4,POINT (29.3586189999948 40.8285630005268)


In [18]:
from shapely import wkt

bus_data["geometry"] = bus_data["geometry"].apply(wkt.loads)

gdf_points = gpd.GeoDataFrame(bus_data, geometry="geometry", crs="EPSG:4326")

gdf_districts = gpd.read_file("../data/istanbul-districts.json")

gdf_points = gdf_points.to_crs(gdf_districts.crs)

result = gpd.sjoin(gdf_points, gdf_districts, how="left", predicate="within")

In [19]:
result.head()

,geometry,index_right,id,name
0,POINT (29.27635 40.8183),5.0,relation/1276010,Tuzla
1,POINT (29.26618 40.81401),5.0,relation/1276010,Tuzla
2,POINT (29.34291 40.8838),5.0,relation/1276010,Tuzla
3,POINT (29.35172 40.82813),5.0,relation/1276010,Tuzla
4,POINT (29.35862 40.82856),5.0,relation/1276010,Tuzla


In [20]:
code_dict = {
    "Adalar": 0,
    "Arnavutköy": 1,
    "Ataşehir": 2,
    "Avcılar": 3,
    "Bağcılar": 4,
    "Bahçelievler": 5,
    "Bakırköy": 6,
    "Başakşehir": 7,
    "Bayrampaşa": 8,
    "Beşiktaş": 9,
    "Beykoz": 10,
    "Beylikdüzü": 11,
    "Beyoğlu": 12,
    "Çatalca": 13,
    "Çekmeköy": 14,
    "Esenler": 15,
    "Esenyurt": 16,
    "Eyüpsultan": 17,
    "Fatih": 18,
    "Gaziosmanpaşa": 19,
    "Güngören": 20,
    "Kadıköy": 21,
    "Kağıthane": 22,
    "Kartal": 23,
    "Küçükçekmece": 24,
    "Maltepe": 25,
    "Pendik": 26,
    "Sancaktepe": 27,
    "Sarıyer": 28,
    "Şile": 29,
    "Şişli": 30,
    "Sultanbeyli": 31,
    "Sultangazi": 32,
    "Tuzla": 33,
    "Ümraniye": 34,
    "Üsküdar": 35,
    "Zeytinburnu": 36,
    "Silivri": 37,
    "Çekmeköy": 38
}

In [21]:
result["name"] = result["name"].apply(lambda x: code_dict[x] if x in code_dict else None).astype("Int64")

result.dropna(subset=["name"], inplace=True)

In [22]:
result.drop(columns=["id", "index_right"], inplace=True)

In [23]:
result.to_csv("../data/bus_data_with_districts.csv", index=False)